In [ ]:
import random
from keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.utils import shuffle

from tqdm import tqdm
from PIL import Image
import skimage.io
import numpy as np
import pandas as pd
#사이킷런
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold #KFOLD

#keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.layers import Activation
from keras.optimizers import Adam
from tensorflow import keras
#efn model
import efficientnet.tfkeras as efn

#etc
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers.experimental import preprocessing

from keras import backend as K


In [ ]:
#GPU 사용하는 만큼만 사용
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)

# Data Call

In [ ]:
def read_image(image_path, resize_ratio=2):
    if not(isinstance(image_path, str)):
        # if tensor with byte string
        image_path = image_path.numpy().decode('utf-8')

    image_level_1 = skimage.io.MultiImage(image_path)[0]
    
    if resize_ratio != 1:
        new_w = int(image_level_1.shape[1]*resize_ratio)
        new_h = int(image_level_1.shape[0]*resize_ratio)
        image_level_1 = cv2.resize(
            image_level_1, (new_w, new_h), interpolation=cv2.INTER_AREA)
    
    return image_level_1


## Training model matrix

In [ ]:
def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall


def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = K.sum(y_pred_yn)

    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())

    # return a single tensor value
    return precision


def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    
    # return a single tensor value
    return _f1score

## Predict Crack tile

In [ ]:
test_PATH = '/home/workspace/data/.train/.task153/data/test/'
test_df = pd.read_csv('/home/workspace/data/.train/.task153/data/test/test.csv',names=['file','tile','label'],sep=',',skiprows=1)
test_N = test_df.shape[0]
test_x = np.empty((test_N,136,136,3), dtype=np.uint8)
for i,path in enumerate(tqdm(test_df.file)): 
    image = read_image(test_PATH + path)
    test_x[i,:,:,:] =  image

In [ ]:

save_model = keras.models.load_model("./Best_EFN_B6_ensemble.h5",
                                              custom_objects={
                                                  'recall': recall,
                                                  'precision': precision,
                                                  'f1score' : f1score}
                                             )
save_model2 = keras.models.load_model("./Best_EFN_B7_V5_resize.h5",
                                              custom_objects={
                                                  'recall': recall,
                                                  'precision': precision,
                                                  'f1score' : f1score}
                                             )

In [ ]:
preds_a = save_model.predict(test_x)
preds_b = save_model2.predict(test_x)

In [ ]:
test_df['label'] = np.argmax(preds_a*0.8+preds_b * 0.2,axis=1)

In [ ]:
test_df.label.to_csv('/home/workspace/user-workspace/prediction/prediction.csv',header=None,index=None)